In [71]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [72]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [73]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [74]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_sensex(f_today):

    days_to_thursday = (4 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [75]:
from datetime import date
from bisect import bisect_left

async def get_expiry(input_date, index_symbol='sensex'):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)    
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [76]:
# bnf_pandas = pd.read_csv('../data/nifty_1hr_tv (2).csv')
# bnf_pandas = pd.read_csv('../data/bnf_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/midcp_select_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/sensex_1hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/crude_4hr_tv.csv')
# bnf_pandas = pd.read_csv('../data/gold_4hr_tv.csv')
# bnf_1min = pd.read_csv('../data/bnf_min.csv')
bnf_1min = pd.read_csv("../data/sensex_min (1).csv")
bnf_1min.columns =  ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
# bnf_1min = bnf_1min[
#     (bnf_1min["datetime"].dt.year >= 2017) & (bnf_1min["datetime"].dt.year <= 2024)
# ]

In [77]:
# # If Stocks Data ...
# bnf_pandas['datetime'] = pd.to_datetime(bnf_pandas['time'])
# bnf_pandas['datetime'] = bnf_pandas['datetime'].dt.tz_localize(None)
# bnf_pandas = bnf_pandas[bnf_pandas['datetime'].dt.year >= 2017]
# bnf_pandas.drop(columns=['time'], inplace=True)
# # bnf_pandas

In [78]:
# bnf = pl.DataFrame(bnf_pandas)
# bnf = bnf.with_columns([pl.col('datetime').alias('index')]).drop('datetime')
# bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# # bnf

In [79]:
bnf_1min.tail()

,index,datetime,o,h,l,c,v
663206,sensex,2025-05-30 15:25:00,81425.3800,81428.3300,81408.7600,81426.2500,0
663207,sensex,2025-05-30 15:26:00,81424.4000,81435.8600,81412.7100,81418.1900,0
663208,sensex,2025-05-30 15:27:00,81420.4200,81430.8200,81407.7000,81421.0100,0
663209,sensex,2025-05-30 15:28:00,81425.7000,81425.7000,81374.9700,81412.6900,0
663210,sensex,2025-05-30 15:29:00,81412.1100,81435.4100,81384.6500,81405.8500,0


In [80]:
from expiries import dict_expiries

In [81]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [82]:
# def calculate_weekly_ranges(df):
#     # Ensure the index is a DatetimeIndex
#     df.index = pd.to_datetime(df.index)
#     # Resample to weekly data to get the high and low for each week
#     weekly_ranges = df.resample("W-WED").agg({"high": "max", "low": "min"})
#     return weekly_ranges

def calculate_weekly_ranges(df, dict_expiries):
    # Ensure datetime index
    df.index = pd.to_datetime(df.index)
    
    # List to hold all weekly range DataFrames
    all_weekly_ranges = []

    expiries = dict_expiries['sensex']
    
    for i in range(len(expiries) - 1):
        current_expiry = pd.to_datetime(expiries[i])
        next_expiry = pd.to_datetime(expiries[i + 1])
        
        # Slice df between current_expiry + 1 day and next_expiry (inclusive)
        start_date = current_expiry + timedelta(days=1)
        end_date = next_expiry
        
        mask = (df.index >= start_date) & (df.index <= end_date)
        df_slice = df.loc[mask]

        # Resample to weekly ranges
        weekly_range = df_slice.resample("W-WED").agg({"high": "max", "low": "min"})
        weekly_range["start_date"] = start_date
        weekly_range["end_date"] = end_date
        
        all_weekly_ranges.append(weekly_range)

    # Combine all into a single DataFrame
    result = pd.concat(all_weekly_ranges)
    print(result)
    return result


async def find_breakouts(df, weekly_ranges):

    results = []
    multiplier_to_range = 0.4

    portfolio_value = 90_00_000
    index_leverage = 9

    index_name = "SENSEX"
    index_str_for_opt = "sensex"

    weekly_ranges["datetime"] = pd.to_datetime(weekly_ranges["datetime"])
    # weekly_ranges = weekly_ranges[:-2]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]

    for i in range(0, len(weekly_ranges)):
        previous_week = weekly_ranges.iloc[i - 1]
        current_week = weekly_ranges.iloc[i]
        print("Current Week : ", current_week["datetime"].date())
        if current_week["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_week_high = weekly_ranges.iloc[i - 1]["high"]
        previous_week_low = weekly_ranges.iloc[i - 1]["low"]
        previous_week_close = weekly_ranges.iloc[i - 1]["close"]

        current_week_open = weekly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current week
        # current_week_data = df[(df.index > weekly_ranges.index[i-1]) & (df.index <= current_week)]
        current_week_data = df.loc[
            (df["datetime"] >= current_week["datetime"])
            & (df["datetime"] <= (current_week["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current Week Data:\n", current_week_data)

        weekly_range = previous_week_high - previous_week_low
        addition_range = multiplier_to_range * weekly_range

        # Check for breakouts
        high_level = current_week_data["open"].iloc[0] + addition_range
        low_level = current_week_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_week_data[current_week_data["high"] >= high_level]
        breakout_low = current_week_data[current_week_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_high_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            low_breach = current_week_data_after_entry[
                current_week_data_after_entry["low"] < previous_week_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')

                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            # exit_date = current_week['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry(
                current_week["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_week_data_after_entry = current_week_data[
                (current_week_data["datetime"] > breakout_low_time)
                & (
                    current_week_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_week_data_after_entry)
            high_breach = current_week_data_after_entry[
                current_week_data_after_entry["high"] > previous_week_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan') 
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "Week": previous_week["datetime"].date(),
                "Week High": previous_week_high,
                "Week Low": previous_week_low,
                "Weekly Range": weekly_range,
                "Current Week Open": current_week_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

async def trade():
    df = bnf_1min
    df = df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close"
    })
    # weekly_ranges = calculate_weekly_ranges(df)
    weekly_ranges = resample(pl.DataFrame(df), "7d", pd.Timedelta(days=1))
    weekly_ranges = weekly_ranges.to_pandas()
    # print(weekly_ranges)
    breakouts = await find_breakouts(df, weekly_ranges)
    breakouts_pandas = pd.DataFrame(breakouts)
    # breakouts_polars = pl.DataFrame(breakouts)
    return breakouts_pandas


tradebook = await trade()
# tradebook

Current Week :  2018-03-02
Current Week :  2018-03-09
Current Week :  2018-03-16
Current Week :  2018-03-23
Current Week :  2018-03-30
Current Week :  2018-04-06
Current Week :  2018-04-13
Current Week :  2018-04-20
Current Week :  2018-04-27
Current Week :  2018-05-04
Current Week :  2018-05-11
Current Week :  2018-05-18
Current Week :  2018-05-25
Current Week :  2018-06-01
Current Week :  2018-06-08
Current Week :  2018-06-15
Current Week :  2018-06-22
Current Week :  2018-06-29
Current Week :  2018-07-06
Current Week :  2018-07-13
Current Week :  2018-07-20
Current Week :  2018-07-27
Current Week :  2018-08-03
Current Week :  2018-08-10
Current Week :  2018-08-17
Current Week :  2018-08-24
Current Week :  2018-08-31


CancelledError: 

In [83]:
tradebook.tail(60)

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs
317,2024-03-01,74245.1719,72591.1406,1654.0313,74187.3516,40%,74848.9641,73525.7391,low,74800,2024-03-15,4,SENSEX2431574800CE,CE,2024-03-11 15:10:00,93.0000,2024-03-15 15:30:00,0.1500,Expiry Closing,92.8500,0.9315,91.9185,1082.8877,100546.1230,99537.4131,1.1172,1.1060
318,2024-03-08,74187.3516,72497.1875,1690.1641,72889.2891,40%,73565.3547,72213.2235,low,73600,2024-03-22,3,SENSEX2432273600CE,CE,2024-03-19 10:03:00,89.2000,2024-03-22 15:30:00,0.1000,Expiry Closing,89.1000,0.8930,88.2070,1100.5435,98058.4239,97075.6386,1.0895,1.0786
319,2024-03-15,72998.0703,71674.4219,1323.6484,72227.8281,40%,72757.2875,71698.3687,high,71700,2024-03-28,6,SENSEX24MAR71700PE,PE,2024-03-22 10:18:00,131.7500,2024-03-28 15:30:00,0.1500,Expiry Closing,131.6000,1.3190,130.2810,1129.7071,148669.4561,147179.3724,1.6519,1.6353
320,2024-03-29,74501.7266,73485.1172,1016.6094,74176.0312,40%,74582.6750,73769.3874,high,73800,2024-04-12,4,SENSEX2441273800PE,PE,2024-04-08 09:33:00,207.3000,2024-04-12 15:30:00,0.1000,Expiry Closing,207.2000,2.0740,205.1260,1097.5610,227414.6341,225138.2927,2.5268,2.5015
321,2024-04-05,75123.2969,73946.9219,1176.3750,74805.8516,40%,75276.4016,74335.3016,low,75300,2024-04-19,7,SENSEX2441975300CE,CE,2024-04-12 12:42:00,174.2000,2024-04-19 15:30:00,0.0500,Expiry Closing,174.1500,1.7425,172.4075,1075.6972,187332.6693,185458.2669,2.0815,2.0606
322,2024-04-12,74951.8828,72365.6719,2586.2109,72014.5391,40%,73049.0235,70980.0547,high,71000,2024-04-26,7,SENSEX24APR71000PE,PE,2024-04-19 13:59:00,159.9000,2024-04-26 15:30:00,0.0500,Expiry Closing,159.8500,1.5995,158.2505,1140.8451,182364.0845,180539.3028,2.0263,2.0060
323,2024-04-26,74812.4297,73616.6484,1195.7813,75017.8203,40%,75496.1328,74539.5078,low,75500,2024-05-10,7,SENSEX2451075500CE,CE,2024-05-03 10:21:00,257.5000,2024-05-10 15:30:00,0.0500,Expiry Closing,257.4500,2.5755,254.8745,1072.8477,276204.6358,273441.5166,3.0689,3.0382
324,2024-05-03,75095.1797,72334.1797,2761.0000,72475.4531,40%,73579.8531,71371.0531,high,71400,2024-05-17,1,SENSEX2451771400PE,PE,2024-05-16 15:00:00,6.9000,2024-05-17 15:30:00,0.1000,Expiry Closing,6.8000,0.0700,6.7300,1134.4538,7714.2857,7634.8739,0.0857,0.0848
325,2024-05-10,73749.4688,71866.0078,1883.4610,73711.3125,40%,74464.6969,72957.9281,high,73000,2024-05-24,1,SENSEX2452473000PE,PE,2024-05-23 09:42:00,26.9000,2024-05-24 15:30:00,0.0500,Expiry Closing,26.8500,0.2695,26.5805,1109.5890,29792.4658,29493.4315,0.3310,0.3277
326,2024-05-17,75499.9062,73459.7969,2040.1093,75335.4531,40%,76151.4968,74519.4094,low,76200,2024-05-31,2,SENSEX24MAY76200CE,CE,2024-05-29 14:45:00,41.2000,2024-05-31 15:30:00,0.0500,Expiry Closing,41.1500,0.4125,40.7375,1062.9921,43742.1260,43303.6417,0.4860,0.4812


In [63]:
# positive = tradebook[tradebook['Points']<0]
# positive['Points'].sum()
tradebook["ROI% w cs"].sum()
# no_value = tradebook[(tradebook['ROI%'] > 0) | (tradebook['ROI%'] < 0)]
# len(no_value)

60.56543567798694

In [64]:
# x = tradebook['Entry Time'].iloc[0].year
tradebook["Entry Time"] = pd.to_datetime(tradebook["Entry Time"])
tradebook["Trade Year"] = tradebook["Entry Time"].dt.year

In [65]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2026),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tradebook
# Iterate over each year
for year in range(2017, 2026):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w cs"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0][
    "ROI% w cs"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0][
    "ROI% w cs"
].mean()
overall_max_drawdown = (
    combined_df_sorted["ROI% w cs"].cumsum()
    - combined_df_sorted["ROI% w cs"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,44,0.0000,NaN,NaN,NaN,NaN
2019,0.0000,51,0.0000,NaN,NaN,NaN,NaN
2020,0.0000,49,0.0000,NaN,NaN,NaN,NaN
2021,0.0000,61,0.0000,NaN,NaN,NaN,NaN
2022,0.0000,56,0.0000,NaN,NaN,NaN,NaN
2023,16.4351,52,38.4615,0.9877,-0.5532,-1.1266,14.5888
2024,21.6138,54,81.4815,0.8905,-1.7566,-7.8344,2.7588
2025,22.5165,19,73.6842,1.8102,-1.4132,-0.1110,202.8595
Overall,60.5654,386.0000,20.2073,1.0805,-1.3173,-10.5497,5.7410


50% MOVE VARIATION FROM CURRENT OPEN, ENTRY AT OPP LEVELS, SL AT PRV. WK H/L

In [40]:
# stats_of_trades = tradebook
# stats_of_trades['Cumulative ROI%'] = stats_of_trades.groupby('Option Type')['ROI%'].cumsum()
# stats_of_trades['Running Max ROI%'] = stats_of_trades.groupby('Option Type')['Cumulative ROI%'].cummax()
# stats_of_trades['Drawdown'] = stats_of_trades['Cumulative ROI%'] - stats_of_trades['Running Max ROI%']
# max_dd_distribution = stats_of_trades.groupby('Option Type')['Drawdown'].min().reset_index()
# max_dd_distribution.rename(columns={'Drawdown': 'Max Drawdown'}, inplace=True)
# roi_distribution = stats_of_trades.groupby('Option Type')['ROI%'].sum().reset_index()
# grouped = pd.merge(roi_distribution, max_dd_distribution, on='Option Type')
# grouped['ROI/DD Ratio'] = grouped['ROI%'] / grouped['Max Drawdown'].abs()
# grouped

In [41]:
# tradebook.to_csv('NIFTY Final TB JJMS RBOS 40pct.csv')
# tradebook2 = pd.read_csv("NIFTY Final TB JJMS RBOS 40pct.csv")
tradebook2 = tradebook
tradebook2["Expiry"] = pd.to_datetime(tradebook2["Expiry"])
tradebook2["Entry Time"] = pd.to_datetime(tradebook2["Entry Time"])
tradebook2["Exit Time"] = pd.to_datetime(tradebook2["Exit Time"])

In [42]:
def round_to_nearest_100(value):
    return 100 * round(value / 100)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 2.75
    index_name = "SENSEX"
    index_str_for_opt = "sensex"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_100(row["Strike"] * (1 + hedge_pct / 100))
            if row["Option Type"] == "CE"
            else round_to_nearest_100(row["Strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    df["Hedge Contract"] = df.apply(
        lambda row: get_option_contract_name2(
            index_name, row["Hedge Strike"], row["Expiry"], row["Option Type"]
        ),
        axis=1,
    )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    for i in range(0, len(df)):
        print(df.iloc[i]["Week"])
        hedge_strike = int(df.iloc[i]["Hedge Strike"])
        hedge_expiry = df.iloc[i]["Expiry"]
        hedge_opt_type = df.iloc[i]["Option Type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["Entry Time"]
        hedge_exit_datetime = df.iloc[i]["Exit Time"]

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=hedge_strike,
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tradebook2)
# tb_with_hedge

2018-03-04
None
2018-03-11
None
2018-03-18
None
2018-03-25
None
2018-04-01
None
2018-04-08
None
2018-04-15
None
2018-04-22
None
2018-04-29
None
2018-05-06
None
2018-05-06
None
2018-05-13
None
2018-05-20
None
2018-05-20
None
2018-05-27
None
2018-06-03
None
2018-06-10
None
2018-06-17
None
2018-06-24
None
2018-07-01
None
2018-07-08
None
2018-07-15
None
2018-07-22
None
2018-07-29
None
2018-08-05
None
2018-08-12
None
2018-08-19
None
2018-08-26
None
2018-09-02
None
2018-09-09
None
2018-09-16
None
2018-09-23
None
2018-10-07
None
2018-10-07
None
2018-10-14
None
2018-10-21
None
2018-11-04
None
2018-11-04
None
2018-11-11
None
2018-11-18
None
2018-11-25
None
2018-12-02
None
2018-12-02
None
2018-12-16
None
2018-12-23
None
2019-01-06
None
2019-01-06
None
2019-01-13
None
2019-01-20
None
2019-01-27
None
2019-02-03
None
2019-02-10
None
2019-02-17
None
2019-02-24
None
2019-03-03
None
2019-03-17
None
2019-03-17
None
2019-03-31
None
2019-04-07
None
2019-04-14
None
2019-04-21
None
2019-04-28
None
2019-05-

In [43]:
# tb_with_hedge = pd.read_csv('JJMS TB w Hedges 1%.csv')
# tb_with_hedge

In [44]:
tb_with_hedge = tb_with_hedge_tmp

In [45]:
tb_with_hedge["Qty"] = tb_with_hedge["Qty"] * 2.75
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL w cs"] + tb_with_hedge["Hedge PnL"]

In [46]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 9000000
tb_with_hedge["Entry Time"] = pd.to_datetime(tb_with_hedge["Entry Time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["Entry Time"].dt.year

In [47]:
# stats_of_trades = tb_with_hedge
# stats_of_trades["Cumulative ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Total ROI%"
# ].cumsum()
# stats_of_trades["Running Max ROI%"] = stats_of_trades.groupby("Option Type")[
#     "Cumulative ROI%"
# ].cummax()
# stats_of_trades["Drawdown"] = (
#     stats_of_trades["Cumulative ROI%"] - stats_of_trades["Running Max ROI%"]
# )
# max_dd_distribution = (
#     stats_of_trades.groupby("Option Type")["Drawdown"].min().reset_index()
# )
# max_dd_distribution.rename(columns={"Drawdown": "Max Drawdown"}, inplace=True)
# roi_distribution = (
#     stats_of_trades.groupby("Option Type")["Total ROI%"].sum().reset_index()
# )
# grouped = pd.merge(roi_distribution, max_dd_distribution, on="Option Type")
# grouped["ROI/DD Ratio"] = grouped["Total ROI%"] / grouped["Max Drawdown"].abs()
# grouped

In [48]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2026),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2026):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,44,0.0000,NaN,NaN,NaN,NaN
2019,0.0000,51,0.0000,NaN,NaN,NaN,NaN
2020,0.0000,49,0.0000,NaN,NaN,NaN,NaN
2021,0.0000,61,0.0000,NaN,NaN,NaN,NaN
2022,0.0000,56,0.0000,NaN,NaN,NaN,NaN
2023,39.5100,52,38.4615,2.4424,-1.5562,-3.1321,12.6146
2024,42.8150,54,81.4815,2.1033,-4.9729,-21.9803,1.9479
2025,54.7922,19,68.4211,4.8596,-2.7943,-1.2804,42.7935
Overall,137.1172,386.0000,19.9482,2.6567,-3.5499,-28.7775,4.7647


In [49]:
tb_with_hedge.to_csv('sensex_rbw.csv')

In [115]:
a = tb_with_hedge["PnL"].sum()
b = tb_with_hedge["Hedge PnL"].sum()
c = tb_with_hedge["Total PnL"].sum()
d = tb_with_hedge["Total ROI%"].sum()
print(a, b, c, d)

10725191.546035036 -3474244.75293266 6917284.3341805115 76.85871482422792


In [116]:
# tb_with_hedge.to_csv('JJMS TB w Hedges 1%.csv')

In [117]:
# # tb_with_hedge['DD%'] = (tb_with_hedge['Total ROI%'].cumsum() - tb_with_hedge['Total ROI%'].cumsum().cummax())
# tb_with_hedge.drop(columns=['Cumulative ROI%', 'Running Max ROI%', 'Drawdown'], inplace=True)
# tb_with_hedge

In [118]:
# tb_with_hedge["ROI%"] = tb_with_hedge["PnL"] * 100 / 1000000

In [119]:
tb_with_hedge.tail(5)

,Week,Week High,Week Low,Weekly Range,Current Week Open,Multiplier to Range,Entry Level Long,Entry Level Short,Break Type,Strike,Expiry,DTE,Contract,Option Type,Entry Time,Entry Price,Exit Time,Exit Price,Remark,Points,Slippage,Points w cs,Qty,PnL,PnL w cs,ROI%,ROI% w cs,Trade Year,Hedge Strike,Hedge Contract,Hedge Entry Price,Hedge Exit Price,Hedge Points,Hedge PnL,Total PnL,Total ROI%
65,2024-11-24,80511.1500,78918.9200,1592.2300,79743.8700,40%,80380.7620,79106.9780,high,79100,2024-12-06,3,SENSEX24120679100PE,PE,2024-12-03 09:15:00,201.7500,2024-12-06 15:30:00,0.0500,Expiry Closing,201.7000,2.0180,199.6820,3413.4008,688482.9330,681594.6903,2.5499,2.5244,2024,77500,SENSEX24120677500PE,40.5000,0.0500,-40.4500,-138072.0607,543522.6296,6.0391
66,2024-12-01,82317.7400,79308.9500,3008.7900,81602.5800,40%,82806.0960,80399.0640,low,82800,2024-12-13,0,SENSEX24121382800CE,CE,2024-12-13 10:21:00,2.3000,2024-12-13 15:30:00,0.0500,Expiry Closing,2.2500,0.0235,2.2265,3260.8696,7336.9565,7260.3261,0.0272,0.0269,2024,84500,SENSEX24121384500CE,1.4000,0.0500,-1.3500,-4402.1739,2858.1522,0.0318
67,2024-12-08,82213.9200,80082.8200,2131.1000,82000.3100,40%,82852.7500,81147.8700,low,82900,2024-12-20,3,SENSEX24122082900CE,CE,2024-12-17 10:20:00,132.4000,2024-12-20 15:30:00,0.2000,Expiry Closing,132.2000,1.3260,130.8740,3256.9361,430566.9481,426248.2509,1.5947,1.5787,2024,84600,SENSEX24122084600CE,20.3000,0.0500,-20.2500,-65952.9554,360295.2955,4.0033
68,2024-12-22,79043.1500,78173.3800,869.7700,78637.5800,40%,78985.4880,78289.6720,high,78300,2025-01-03,4,SENSEX2510378300PE,PE,2024-12-30 11:47:00,NaN,2024-12-30 13:54:00,NaN,SL Hit,NaN,NaN,NaN,3448.2759,NaN,NaN,NaN,NaN,2024,76700,SENSEX2510376700PE,0.0000,0.0000,0.0000,0.0000,NaN,NaN
69,2024-12-22,79043.1500,78173.3800,869.7700,78637.5800,40%,78985.4880,78289.6720,low,79000,2025-01-03,4,SENSEX2510379000CE,CE,2024-12-30 13:36:00,NaN,2025-01-03 15:30:00,NaN,Expiry Closing,NaN,NaN,NaN,3417.7215,NaN,NaN,NaN,NaN,2024,80600,SENSEX2510380600CE,0.0000,0.0000,0.0000,0.0000,NaN,NaN


In [120]:
stats_df9 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df9.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df9["Total ROI"].sum()
overall_total_trades = stats_df9["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df9.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df9

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,18.8665,17,70.5882,2.0148,-1.0623,-2.2466,8.3978
2024,57.9922,53,77.3585,2.4083,-5.8214,-23.4470,2.4733
Overall,76.8587,70.0000,77.1429,2.3192,-3.8384,-23.4470,3.2780


In [121]:
# tradebook["Cumulative ROI%"] = tradebook["ROI% w cs"].cumsum()
# tradebook["Running Max ROI%"] = tradebook["Cumulative ROI%"].cummax()
# tradebook["Drawdown"] = tradebook["Cumulative ROI%"] - tradebook["Running Max ROI%"]
# tradebook

In [122]:
tradebook.to_csv("RBW_SENSEX.csv", index=False)

In [118]:
async def find_breakouts_monthly(df, monthly_ranges):

    results = []
    multiplier_to_range = 0.5

    portfolio_value = 10_00_000
    index_leverage = 10

    index_name = "NIFTY"
    index_str_for_opt = "nifty"

    monthly_ranges["datetime"] = pd.to_datetime(monthly_ranges["datetime"])
    monthly_ranges = monthly_ranges[:-1]
    # print(weekly_ranges)

    dates_to_avoid = [
        # dt.date(2017, 1, 23),
        # dt.date(2017, 3, 13),
        # dt.date(2017, 4, 10),
        # dt.date(2017, 12, 11),
        # dt.date(2019, 3, 25),
        # dt.date(2019, 4, 1),
        # dt.date(2019, 9, 9),
        # dt.date(2020, 4, 6),
        # dt.date(2020, 10, 26),
    ]
    
    for i in range(1, len(monthly_ranges)):
        previous_mmonth = monthly_ranges.iloc[i - 1]
        current_mmonth = monthly_ranges.iloc[i]
        print("Current mmonth : ", current_mmonth["datetime"].date())
        if current_mmonth["datetime"].date() in dates_to_avoid:
            print("Date Avoided")
            continue
        previous_mmonth_high = monthly_ranges.iloc[i - 1]["high"]
        previous_mmonth_low = monthly_ranges.iloc[i - 1]["low"]
        previous_mmonth_close = monthly_ranges.iloc[i - 1]["close"]

        current_mmonth_open = monthly_ranges.iloc[i]["open"]

        # Filter the 1-minute data for the current mmonth
        # current_mmonth_data = df[(df.index > mmonthly_ranges.index[i-1]) & (df.index <= current_mmonth)]
        current_mmonth_data = df.loc[
            (df["datetime"] >= current_mmonth["datetime"])
            & (df["datetime"] <= (current_mmonth["datetime"] + pd.Timedelta(days=7)))
        ]
        # print("Current mmonth Data:\n", current_mmonth_data)

        monthly_range = previous_mmonth_high - previous_mmonth_low
        addition_range = multiplier_to_range * monthly_range

        # Check for breakouts
        high_level = current_mmonth_data["open"].iloc[0] + addition_range
        low_level = current_mmonth_data["open"].iloc[0] - addition_range

        # high_level = current_week_data['open'].iloc[0] + addition_range
        # low_level = current_week_data['open'].iloc[0] - addition_range
        # print(current_week)
        # print("High Level:\n", high_level, "\nLow Level:\n", low_level)

        breakout_high = current_mmonth_data[currentmmonthk_data["high"] >= high_level]
        breakout_low = current_mmonth_data[current_mmonth_data["low"] <= low_level]

        if not breakout_high.empty:
            # print('Breakout High')
            breakout_high_time = breakout_high.iloc[0]["datetime"]
            # print('High Break At :', breakout_high_time)
            atm_strike = int(round(low_level / 100) * 100)
            # exit_date = currentmmonthk['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_sensex(
                current_mmonth["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_high_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="PE",
            )
            pe_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_high_time.date(),
                end_date=expiry,
                start_time=breakout_high_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="P",
            )
            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.to_pandas()
                entry_price = pe_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_mmonth_data_after_entry = current_mmonth_data[
                (current_mmonth_data["datetime"] > breakout_high_time)
                & (
                    current_mmonth_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_mmonth_data_after_entry)
            low_breach = current_mmonth_data_after_entry[
                current_mmonth_data_after_entry["low"] < previous_mmonth_low
            ]
            # print(low_breach)
            if len(low_breach) != 0:
                exit_time = low_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                # print(pe_df)
                if not math.isnan(entry_price):
                    # print((pe_df[(pe_df['datetime'].dt.date == exit_time.date()) & (pe_df['datetime'].dt.hour >= exit_time.hour) & (pe_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = pe_df.loc[
                        (pe_df["datetime"].dt.date >= exit_time.date())
                        & (pe_df["datetime"].dt.hour >= exit_time.hour)
                        & (pe_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Friday Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = pe_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)
            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "mmonth": previous_mmonth["datetime"].date(),
                "mmonth High": previous_mmonth_high,
                "mmonth Low": previous_mmonth_low,
                "Monthly Range": monthly_range,
                "Current mmonth Open": current_mmonth_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "high",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_high_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

        if not breakout_low.empty:
            # print('Breakout Low')
            breakout_low_time = breakout_low.iloc[0]["datetime"]
            # print('Low Break At :', breakout_low_time)
            atm_strike = int(round(high_level / 100) * 100)
            # exit_date = current_mmonth['datetime'].date() + dt.timedelta(days=5)
            expiry = await get_expiry_sensex(
                current_mmonth["datetime"].date() + dt.timedelta(days=5)
            )
            dte = (expiry - breakout_low_time.date()).days
            contract = await get_option_contract_name(
                symbol=index_name,
                strike=atm_strike,
                expiry=expiry,
                opt_type="CE",
            )
            ce_df = await fetch_data(
                index=index_str_for_opt,
                start_date=breakout_low_time.date(),
                end_date=expiry,
                start_time=breakout_low_time.time(),
                end_time=dt.time(15, 20),
                expiry=expiry,
                strike=atm_strike,
                asset_class="C",
            )
            # print(len(ce_df))
            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.to_pandas()
                entry_price = ce_df.iloc[0]["c"]
            else:
                entry_price = float("nan")

            # Exit Scenario
            current_mmonth_data_after_entry = current_mmonth_data[
                (current_mmonth_data["datetime"] > breakout_low_time)
                & (
                    current_mmonth_data["datetime"].dt.date
                    <= expiry - dt.timedelta(days=0)
                )
            ]
            # print(current_mmonth_data_after_entry)
            high_breach = current_mmonth_data_after_entry[
                current_mmonth_data_after_entry["high"] > previous_mmonth_high
            ]
            # print(high_breach)
            if len(high_breach) != 0:
                exit_time = high_breach.iloc[0]["datetime"]
                # print(exit_time)
                remark = "SL Hit"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    # print(exit_time.date(), exit_time.hour, exit_time.minute)
                    # print((ce_df[(ce_df['datetime'].dt.date == exit_time.date()) & (ce_df['datetime'].dt.hour >= exit_time.hour) & (ce_df['datetime'].dt.minute >= exit_time.minute)]).head(3))
                    filtered_df = ce_df.loc[
                        (ce_df["datetime"].dt.date >= exit_time.date())
                        & (ce_df["datetime"].dt.hour >= exit_time.hour)
                        & (ce_df["datetime"].dt.minute >= exit_time.minute),
                        "c"]
                    exit_price = filtered_df.iloc[0] if not filtered_df.empty else float('nan')
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            else:
                exit_time = dt.datetime.combine(
                    expiry - dt.timedelta(days=0), dt.time(15, 30)
                )
                remark = "Expiry Closing"
                # print(remark, exit_time)
                if not math.isnan(entry_price):
                    exit_price = ce_df.iloc[-1]["c"]
                else:
                    exit_price = float("nan")
                    # print(exit_price)
            # print(exit_time)
            # print(exit_price)
            # print(contract)

            qty = portfolio_value * index_leverage / atm_strike
            slippage = 0.01 * (entry_price + exit_price)
            pnl = qty * (entry_price - exit_price)
            final_pnl = qty * (entry_price - exit_price - slippage)
            unit = {
                "month": previous_month["datetime"].date(),
                "month High": previous_month_high,
                "month Low": previous_month_low,
                "Monthly Range": monthly_range,
                "Current month Open": current_month_open,
                "Multiplier to Range": f"{int(multiplier_to_range * 100)}%",
                "Entry Level Long": high_level,
                "Entry Level Short": low_level,
                "Break Type": "low",
                "Strike": atm_strike,
                "Expiry": expiry,
                "DTE": dte,
                "Contract": contract,
                "Option Type": contract[-2:],
                "Entry Time": breakout_low_time,
                "Entry Price": entry_price,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Remark": remark,
                "Points": entry_price - exit_price,
                "Slippage": slippage,
                "Points w cs": (entry_price - exit_price) - slippage,
                "Qty": qty,
                "PnL": pnl,
                "PnL w cs": final_pnl,
                "ROI%": (pnl * 100 / portfolio_value),
                "ROI% w cs": (final_pnl * 100 / portfolio_value),
            }
            # print(unit)
            results.append(unit)

    # print(results)
    return results

In [124]:
async def trade():
    bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
    print(bnf_1min.head())
    df = bnf_1min
    # weekly_ranges = calculate_weekly_ranges(df)
    monthly_ranges = resample(pl.DataFrame(df), "1mo", pd.Timedelta(days=0))
    monthly_ranges = monthly_ranges.to_pandas()
    print(monthly_ranges)
    # breakouts = await find_breakouts_monthly(df, monthly_ranges)
    # breakouts_pandas = pd.DataFrame(breakouts)
    # # breakouts_polars = pl.DataFrame(breakouts)
    # return breakouts_pandas


tradebook = await trade()
# tradebook

             datetime       open       high        low      close
0 2018-03-08 09:15:00 33219.5200 33292.2200 33219.5200 33286.7100
1 2018-03-08 09:16:00 33283.4200 33306.6300 33275.7700 33306.0200
2 2018-03-08 09:17:00 33279.8700 33279.8700 33256.5300 33258.1200
3 2018-03-08 09:18:00 33264.2600 33288.4800 33264.2600 33282.4700
4 2018-03-08 09:19:00 33271.1500 33271.1500 33264.9200 33266.2300
     datetime       open       high        low      close
0  2018-03-01 33219.5200 34068.3000 32483.8400 33015.6900
1  2018-04-01 33062.0900 35212.9500 32972.6300 35173.0300
2  2018-05-01 35347.5900 35993.5300 34302.8900 35287.4900
3  2018-06-01 35323.0300 35875.7500 34785.0200 35402.9100
4  2018-07-01 35536.3800 37644.1700 35106.5700 37599.4600
5  2018-08-01 37622.6300 38975.2900 37128.9900 38613.8000
6  2018-09-01 38918.4500 38918.4500 35987.2300 36222.3100
7  2018-10-01 36274.4900 36615.2300 33292.3500 34370.3900
8  2018-11-01 34650.6300 36389.2200 34303.8700 36240.1800
9  2018-12-01 36406.4100